# ReAct Zero-Shot Custom Search Engine with LangChain and Google's Programmable Search Engine

This uses Google's programmable search engine to search Riot's official Valorant sites and [GamerGirl](https://gamergirl.gg/) for supporting information, since Riot has not historically been the best at making certain things accessible.

## Setup

Make sure to set up your own Programmable Search Engine and add the `GOOGLE_CSE_ID` variable to the [.env](../../.env) file before running this example. If you set that variable after starting your development environment, you'll need to restart (`make stop; make start;`).

## Configure variables

In [1]:
import os

# ****************** [START] Google Cloud project settings ****************** #
project =  os.getenv('GCP_PROJECT')
location = os.environ.get('GCP_REGION', 'us-central1')

google_api_key =  os.getenv('GOOGLE_API_KEY')
google_cse_id =  os.getenv('GOOGLE_CSE_ID')
# ******************* [END] Google Cloud project settings ******************* #


# *********************** [START] LLM parameter config ********************** #
# Vertex AI model to use for the LLM
model_name='text-bison@002'

# maximum number of model responses generated per prompt
candidate_count = 1

# determines the maximum amount of text output from one prompt.
# a token is approximately four characters.
max_output_tokens = 1024
# max_output_tokens = 256

# temperature controls the degree of randomness in token selection.
# lower temperatures are good for prompts that expect a true or
# correct response, while higher temperatures can lead to more
# diverse or unexpected results. With a temperature of 0 the highest
# probability token is always selected. for most use cases, try
# starting with a temperature of 0.2.
temperature = 0.2

# top-p changes how the model selects tokens for output. Tokens are
# selected from most probable to least until the sum of their
# probabilities equals the top-p value. For example, if tokens A, B, and C
# have a probability of .3, .2, and .1 and the top-p value is .5, then the
# model will select either A or B as the next token (using temperature).
# the default top-p value is .8.
top_p = 0.8

# top-k changes how the model selects tokens for output.
# a top-k of 1 means the selected token is the most probable among
# all tokens in the model’s vocabulary (also called greedy decoding),
# while a top-k of 3 means that the next token is selected from among
# the 3 most probable tokens (using temperature).
top_k = 40

# how verbose the llm and langchain agent is when thinking
# through a prompt. you're going to want this set to True
# for development so you can debug its thought process
verbose = True
# *********************** [END] LLM parameter config ************************ #


# ********************** [START] Configuration Checks *********************** #
if not project or not google_api_key or not google_cse_id:
    raise Exception(f"""All of the following environment variables must be set:
                    - GCP_PROJECT: {project}
                    - GOOGLE_API_KEY: {google_api_key}
                    - GOOGLE_CSE_ID: {google_cse_id}
                    """)
# *********************** [END] Configuration Checks ************************ #


## Import and Initialize Vertex AI Client

This will complain about not having cuda drivers and the GPU not being used. You can safely ignore that. If you want to use the GPU, that's possible in Linux with Docker, but you'll need to set up a non-containerized development environment to use GPUs with MacOS.

In [2]:
from google.cloud import aiplatform
import vertexai

vertexai.init(project=project, location=location)

print(f"Vertex AI SDK version: {aiplatform.__version__}")


2023-12-17 02:22:23.417435: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-17 02:22:23.419137: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 02:22:23.438270: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 02:22:23.438302: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 02:22:23.438317: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Vertex AI SDK version: 1.38.1


## Import LangChain

This doesn't actually initialize anything, it just lets us print the version.

In [3]:
import langchain

print(f"LangChain version: {langchain.__version__}")


LangChain version: 0.0.350


## Configure LLM with Vertex AI

In [4]:
from langchain.llms import VertexAI

llm = VertexAI(
    model_name=model_name,
    max_output_tokens=max_output_tokens,
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    verbose=verbose,
)


## Create Help, FAQ, Support, and Tutorials Search Engine Tool

In [5]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

google_search_valorant = GoogleSearchAPIWrapper(
  google_cse_id=google_cse_id,
  google_api_key=google_api_key,
  # https://python.langchain.com/docs/integrations/tools/google_search#number-of-results
  # You can use the k parameter to set the number of results
  k=5,
)

google_search_engine_valorant = Tool(
    name="Valorant Help, Support, Tutorials, and Frequently Asked Questions Search",
    description=f"""
Search the help, support, tutorials, and frequently asked questions for Valorant.
This tool should be used if a question is about Valorant.
""".strip(),
    func=google_search_valorant.run,
)


## Build tool chain

Add our Valorant search engine to the agents available tools.

In [6]:
tools = [
  google_search_engine_valorant,
]


## Initialize Agent

In [7]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent, AgentExecutor

# initialize ReAct agent
react = initialize_agent(
  tools,
  llm,
  # the wikipedia tool is structured, so this agent type gives
  # it the ability to use both wikipedia and our simpler tools
  agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
  verbose=True,
  # https://python.langchain.com/docs/modules/agents/how_to/max_time_limit
  max_execution_time=60,
  # By default, the early stopping uses the force method which
  # just returns that constant string. Alternatively, you could
  # specify the generate method which then does one FINAL pass
  # through the LLM to generate an output.
  early_stopping_method="generate",
)

agent_executor = AgentExecutor.from_agent_and_tools(
  agent=react.agent,
  tools=tools,
  verbose=True,
  handle_parsing_errors=True,
)


## Ask the agent some questions about recent Valorant updates

In [17]:
query = "What updates were released in the latest Valorant patch?"
agent_executor.run(query)




> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "Valorant Help, Support, Tutorials, and Frequently Asked Questions Search",
  "action_input": "valorant latest patch notes"
}
```

Observation: VALORANT Patch Notes 7.12. It's the last Patch for 2023 and Episode 7! See Article ... Riot Games presents VALORANT: a 5v5 character-based tactical FPS where precise gunplay meets unique agent abilities. Learn about VALORANT and its stylish ... Riot Games presents VALORANT: a 5v5 character-based tactical FPS where precise gunplay meets unique agent abilities. Learn about VALORANT and its stylish cast. Nov 14, 2023 ... Deadlock · Anyone caught by GravNet will now be debuffed indefinitely until they remove it · Anyone caught by GravNet will now be affected by ... Oct 31, 2023 ... Aside from all the new things released at the beginning of a new Act, we also have some Agent updates below. Patch_Notes_Highlights_7_09.jpg ...
Thought: Action:
```
{
  "action": "Final Answer",
  "actio

'The latest Valorant patch, version 7.12, was released on November 14, 2023. It includes a new map called "Deadlock", agent updates, and various bug fixes and improvements.'